## PREMIER LEAGUE MATCH PREDICTION NOTEBOOK

The main steps of this project are as follows:

Data Collection: I will gather match data, including advanced statistics, for Premier League games. This data will be the foundation for training and evaluating my prediction models.

Data Preprocessing: Before feeding the data into my models, I'll clean, transform, and engineer relevant features to ensure the best possible performance.

Model Building: Using machine learning algorithms, I'll construct separate prediction models for the home team's score and the away team's score. These models will learn from historical data to make predictions for future matches.

Model Evaluation: I will assess the performance of my prediction models using various evaluation metrics, ensuring they are reliable and capable of generalizing to new, unseen data.

Match Prediction: Armed with my trained models, I'll predict the scores of upcoming Premier League matches based on their respective team statistics.

While I strive for accurate predictions, I am well aware that football is a sport filled with surprises, and unforeseen events can always occur during matches. My primary purpose in this project is to explore the application of machine learning techniques to sports analytics and gain a better understanding of the factors that influence match outcomes.

## LIBRAIRIES IMPORT

I use essential libraries such as Pandas and NumPy for data manipulation and analysis. Seaborn and Matplotlib are employed for data visualization, enabling us to gain insights and understand patterns in the data. I also utilize Scikit-learn's RandomForestRegressor, a powerful machine learning algorithm, to build separate prediction models for the home team's score and the away team's score, I use GridSearch to find the best parameters for the model. Feature scaling is achieved with MinMaxScaler to enhance model performance. Finally, I measure the efficiency of the models with the mean squared error metric and split our data into training and testing sets using train_test_split from Scikit-learn.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

## DATA IMPORT 

In this project, we have three important datasets containing the necessary information for predicting Premier League match outcomes:

df_results: This dataset, named "2023_matchday_results.csv," contains the real results of Premier League matches for the 2023 season. Each row in this dataset represents a match and includes details such as the date, teams playing, and the actual scores. It serves as the ground truth data against which we will evaluate the performance of our prediction models.

df_home_results: This dataset, named "2023_home_teams_stats.csv," contains advanced statistics for each home team in the 2023 Premier League season. Each row corresponds to a home team's performance in a specific match. These statistics provide insights into how well each team performs when playing at their home ground.

df_away_results: This dataset, named "2023_away_teams_stats.csv," contains advanced statistics for each away team in the 2023 Premier League season. Similar to df_home_results, each row represents an away team's performance in a particular match. These statistics help us understand the performance of teams when playing away from their home ground.

data from : https://www.kaggle.com/datasets/afnanurrahim/premier-league-2022-23

In [ ]:
df_away_results = pd.read_csv("./2023_away_teams_stats.csv")
df_home_results = pd.read_csv("./2023_home_teams_stats.csv")
df_results = pd.read_csv("./2023_matchday_results.csv")

## DATA PREPROCESSING

In the data preprocessing phase, I cleaned and prepared the datasets for prediction models by dropping irrelevant columns, renaming some columns for clarity, and handling missing values. I also converted percentage columns to integer format for easier processing. These steps ensure the data is suitable for training accurate Premier League match prediction models.


In [ ]:
columns_to_drop = ["Unnamed: 0", "teams.away.winner", "teams.home.winner", "fixture.id", "fixture.date", "teams.away.id", "teams.home.id"]
df_results = df_results.drop(columns_to_drop, axis=1)
df_results.rename(columns={"teams.home.name": "Home team name", "teams.away.name": "Away team name"}, inplace=True)

df_home_results = df_home_results.drop('expected_goals', axis=1)
df_away_results = df_away_results.drop("expected_goals", axis=1)

df_home_results['Passes %'] = df_home_results['Passes %'].str.replace('%', '').astype(int)
df_away_results['Passes %'] = df_away_results['Passes %'].str.replace('%', '').astype(int)

df_home_results['Ball Possession'] = df_home_results['Ball Possession'].str.replace('%', '').astype(int)
df_away_results['Ball Possession'] = df_away_results['Ball Possession'].str.replace('%', '').astype(int)


home_columns = df_home_results.columns
away_columns = df_away_results.columns

for column in home_columns:
    df_home_results[column] = df_home_results[column].fillna(0)

for column in away_columns:
        df_away_results[column] = df_away_results[column].fillna(0)

df_away_results.head()

# Merge the data

In this step, I combined home and away team statistics with the actual match results to create a comprehensive dataset (df_final) for Premier League match prediction.
I calculated the mean statistics for each home team and away team from separate datasets (df_home_results and df_away_results).
Then, I merged home team statistics with the match results (df_results) to create add_home_team_stats, adding "Home" as a prefix to the column names.
Next, I merged the calculated away team statistics with add_home_team_stats to obtain add_away_team_stats, including both home and away team statistics for each match.
I renamed the columns for away team statistics with "Away" as a prefix and changed "goals.home" and "goals.away" to "Home Goals" and "Away Goals" for clarity.
The resulting df_final dataset includes all relevant statistics and actual match results for each game, making it suitable for training prediction models

In [ ]:
df_teams_home_mean_stats = pd.DataFrame()
for stat in home_columns[3:]:
    df_teams_home_mean_stats[stat] = df_home_results.groupby('Home team name')[stat].mean()

df_teams_away_mean_stats = pd.DataFrame()
for stat in away_columns[3:]:
    df_teams_away_mean_stats[stat] = df_away_results.groupby('away team name')[stat].mean()

df_teams_away_mean_stats.reset_index(inplace=True)
df_teams_away_mean_stats.rename(columns={'index': 'Away team name'}, inplace=True)
df_teams_away_mean_stats.rename(columns={'away team name': "Away team name"}, inplace=True)

df_teams_home_mean_stats.reset_index(inplace=True)
df_teams_home_mean_stats.rename(columns={'index': 'Home team name'}, inplace=True)


df_teams_away_mean_stats.head()


In [ ]:
add_home_team_stats = pd.merge(df_results, df_teams_home_mean_stats, on='Home team name', how="inner")

for stat in add_home_team_stats.columns[4:]:
    add_home_team_stats.rename(columns={stat: "Home " + stat}, inplace=True)
add_home_team_stats.head()

In [ ]:
add_away_team_stats = pd.merge(add_home_team_stats, df_teams_away_mean_stats, on="Away team name", how="inner")


df_final = add_away_team_stats.rename(columns={"goals.home": "Home Goals", "goals.away": "Away Goals"})

for stat in df_final.columns[20:]:
    df_final.rename(columns={stat: "Away " + stat}, inplace=True)

df_final.head()


The heatmap visualizes the correlation matrix of the df_final dataset, showing the relationships between different variables. Positive correlations appear as lighter colors, negative correlations as darker colors, and no correlation as neutral colors. This helps identify significant features influencing match outcomes.

In [ ]:
plt.figure(figsize=(34,34))
sns.heatmap(df_final.corr(), annot=True, fmt=".2f")
plt.show()

The MinMaxScaler is applied to scale the advanced statistics in the df_final DataFrame to a range between 0 and 1. This feature scaling ensures that all statistics have a common scale, preventing dominance by features with larger values. 

In [ ]:
scaler = MinMaxScaler()
columns_to_scale = df_final.columns[4:]

# Fit and transform the selected columns using Min-Max scaling
df_final[columns_to_scale] = scaler.fit_transform(df_final[columns_to_scale])

# Display the scaled DataFrame
df_final.head()

In this section, I split the data into training and testing sets for both the home team's score and the away team's score. The target variables (y_home and y_away) are extracted, and the input features (X) are defined by dropping irrelevant columns. The train_test_split function from Scikit-learn is used to create separate training and testing sets with a test size of 20% and a random state of 42 to ensure reproducibility. These sets will be used to train and evaluate the prediction models for both home and away team scores

In [ ]:
y_home = df_final["Home Goals"]
y_away = df_final["Away Goals"]
X = df_final.drop(["Home Goals", "Away Goals", "Home team name", "Away team name"], axis=1)

X_home_train, X_home_test, y_home_train, y_home_test = train_test_split(X,y_home, test_size= 0.2, random_state=42)
X_away_train, X_away_test, y_away_train, y_away_test = train_test_split(X,y_away, test_size= 0.2, random_state=42)


## MODEL

In this section, we perform hyperparameter tuning using GridSearchCV to find the optimal hyperparameters for our RandomForestRegressor models for predicting home and away team scores. We explore different combinations of "n_estimators," "max_depth," and "min_samples_split" using cross-validation to select the best configuration. The tuned models (best_model_home and best_model_away) will be used for more accurate predictions in our Premier League match prediction project.

In [ ]:
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

### HOME
model_home = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=model_home, param_grid=param_grid, cv=3, scoring = 'neg_mean_squared_error')
grid_search.fit(X_home_train, y_home_train)
best_model_home = grid_search.best_estimator_

### AWAY
model_away = RandomForestRegressor(random_state=42)


grid_search = GridSearchCV(estimator=model_away, param_grid=param_grid, cv=3, scoring = 'neg_mean_squared_error')
grid_search.fit(X_away_train, y_away_train)
best_model_away = grid_search.best_estimator_


In this section, I use the tuned RandomForestRegressor models (best_model_home and best_model_away) to make predictions for the home team's scores (y_home_pred) and the away team's scores (y_away_pred) on the test data (X_home_test and X_away_test, respectively).

These predictions represent the model's estimates for the scores of Premier League matches based on the advanced statistics of the home and away teams. By comparing these predictions with the actual test data (y_home_test and y_away_test), we can evaluate the accuracy and performance of our prediction models in capturing the match outcomes.

In [ ]:
y_home_pred = best_model_home.predict(X_home_test)
y_away_pred = best_model_away.predict(X_away_test)

The learning curves plot shows the training and cross-validation scores against the size of the training data. It helps in understanding how the model's performance improves or stabilizes as more data is used for training. If the training and cross-validation scores converge and remain stable with increasing data size, it suggests that the model might not benefit significantly from additional data and may have reached its capacity. On the other hand, if the training and cross-validation scores diverge, it indicates potential overfitting or underfitting issues that need to be addressed.

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(model, X, y, title):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 10))

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color='blue')
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color='orange')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training Score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='orange', label='Cross-Validation Score')
    plt.xlabel('Training Data Size')
    plt.ylabel('Score')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Call the function for the home team prediction
plot_learning_curve(best_model_home, X, y_home, 'Learning Curve for Home Team Prediction')

# Call the function for the away team prediction
plot_learning_curve(best_model_away, X, y_away, 'Learning Curve for Away Team Prediction')


The code below evaluates the performance of the prediction models by calculating the Mean Squared Error (MSE) for both home and away team scores. The results are printed, and the predicted and actual values are converted into NumPy arrays for further analysis.

In [ ]:
mse_home = mean_squared_error(y_home_test, y_home_pred)
mse_away = mean_squared_error(y_away_test, y_away_pred)

print(f"{mse_home} {mse_away}")

y_home_pred_array = np.array(y_home_pred)
y_home_test_array = np.array(y_home_test)
y_away_pred_array = np.array(y_away_pred)
y_away_test_array = np.array(y_away_test)

The code below generates scatter plots to visualize the efficiency of the prediction results for home and away team scores. The scatter points' proximity to the orange line, representing perfect predictions, indicates the model's accuracy. Deviation from the line highlights prediction errors. These plots offer a quick and visual assessment of the model's performance in predicting match outcomes, aiding in identifying areas for improvement.

In [ ]:
plt.figure()

# First subplot for y_home
plt.subplot(2, 1, 1)
plt.scatter(y_home_test, y_home_pred_array)
plt.plot([x for x in np.linspace(0, 6)], np.linspace(0, 6), 'orange')
plt.xlabel('y_home_test')
plt.ylabel('y_home_pred_array')
plt.grid()

# Second subplot for y_away
plt.subplot(2, 1, 2)
plt.scatter(y_away_test, y_away_pred_array)
plt.plot([x for x in np.linspace(0, 6)], np.linspace(0, 6), 'orange')
plt.xlabel('y_away_test')
plt.ylabel('y_away_pred_array')
plt.grid()

plt.tight_layout()
plt.show()
plt.clf()


The code generates residual plots for home and away team predictions. Residuals represent the differences between actual and predicted values. The plots help assess the model's accuracy, with well-fitted models showing residuals scattered randomly around zero. 

In [ ]:
def plot_residual(actual, predicted, title):
    # Calculate the residuals (difference between actual and predicted values)
    residuals = actual - predicted

    # Create a residual plot
    plt.figure(figsize=(8, 6))
    plt.scatter(predicted, residuals)
    plt.axhline(y=0, color='red', linestyle='--')  # Add a horizontal line at y=0 for reference
    plt.xlabel('Predicted Scores')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.title(title)
    plt.grid()
    plt.tight_layout()
    plt.show()

# Create residual plots for home team predictions
plot_residual(y_home_test, y_home_pred_array, 'Home Team Residual Plot')

# Create residual plots for away team predictions
plot_residual(y_away_test, y_away_pred_array, 'Away Team Residual Plot')


This code will generate two separate bar plots, one for the home team's feature importances and another for the away team's feature importances. The feature importance plot helps in understanding which advanced statistics have the most significant impact on the match scores. By visualizing the importance scores, one can identify the key features that play a crucial role in predicting match outcomes for both the home and away teams.

In [ ]:
importances_home = best_model_home.feature_importances_
importances_away = best_model_away.feature_importances_

df_features = pd.DataFrame({'Feature': X.columns, 'Home Team Importance': importances_home, 'Away Team Importance': importances_away})

df_features = df_features.sort_values(by='Home Team Importance', ascending=False)

plt.figure(figsize=(10, 15))

plt.subplot(2, 1, 1)
plt.bar(df_features['Feature'], df_features['Home Team Importance'])
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance for Home Team Prediction')

plt.subplot(2, 1, 2)
plt.bar(df_features['Feature'], df_features['Away Team Importance'])
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance for Away Team Prediction')

plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
def plot_distribution(actual, predicted, title):
    plt.figure(figsize=(8, 6))
    sns.histplot(actual, kde=True, label='Actual', color='blue')
    sns.histplot(predicted, kde=True, label='Predicted', color='orange')
    plt.xlabel('Scores')
    plt.ylabel('Density')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_distribution(y_home_test, y_home_pred_array, 'Home Team Score Distribution')
plot_distribution(y_away_test, y_away_pred_array, 'Away Team Score Distribution')


The code below compares the predicted match outcomes with the actual outcomes to evaluate the efficiency of the prediction models. It calculates the percentage of predicted home wins, draws, and away wins and compares them to the percentage of actual outcomes in the dataset. The printed results provide an overview of how well the models perform in predicting match results and offer insights into their accuracy and effectiveness. 
The choice of 0.5 as the difference threshold allows us to consider scores like 0.8 - 1.2 as draws, as goals are given as floats for better precision. Without considering this threshold, the number of draws would be null.

In [ ]:
difference_to_consider_a_draw = 0.5

nb_win_home_pred = 0
nb_win_away_pred = 0
nb_draw_pred = 0

length_pred = y_home_pred_array.shape[0]
for i in range(length_pred):
    if abs(y_home_pred_array[i] - y_away_pred_array[i]) < difference_to_consider_a_draw:
        nb_draw_pred += 1
    elif  y_home_pred_array[i] > y_away_pred_array[i]:
        nb_win_home_pred +=1   
    else:
        nb_win_away_pred +=1


percentage_win_home_pred = nb_win_home_pred / length_pred
percentage_win_away_pred = nb_win_away_pred / length_pred
percentage_draw_pred = nb_draw_pred / length_pred


nb_win_home = 0
nb_win_away= 0
nb_draw = 0

length = df_final.shape[0]
for i in range(length):
    if abs(y_home[i] - y_away[i]) < difference_to_consider_a_draw:
        nb_draw += 1
    elif  y_home[i] > y_away[i]:
        nb_win_home +=1    
    else:
        nb_win_away +=1

percentage_win_home = nb_win_home / length
percentage_win_away = nb_win_away / length
percentage_draw = nb_draw / length


sep = "-"*10

print(f"""Predictions Home Win %  : {percentage_win_home_pred} {sep} Real Home Win % : {percentage_win_home}
Predictions Draw % : {percentage_draw_pred} {sep} Real Draw % : {percentage_draw}
Predictions Away Win % : {percentage_win_away_pred} {sep} Real Away Win : {percentage_win_away}""")

The code below defines two functions for match prediction: match_prediction, which predicts the scores of individual matches using tuned RandomForestRegressor models, and export_all_predictions, which exports all match predictions to a text file. The match_prediction function takes home and away team names as input and returns a formatted string with the predicted scores. The export_all_predictions function generates predictions for all possible combinations of home and away teams and saves them to a file named "all_predictions.txt". This enables easy access to match predictions for further analysis or sharing.

In [ ]:
def match_prediction(home_team: str, away_team: str):

    match = df_final.loc[(df_final['Home team name'] == home_team) & (df_final['Away team name'] == away_team)]
    match = match.drop(["Home Goals", "Away Goals", "Home team name", "Away team name"], axis=1)

    home_goals_pred = best_model_home.predict(match)
    away_goals_pred = best_model_away.predict(match)

    prediction = f"{home_team} {round(home_goals_pred[0],2)} - {round(away_goals_pred[0],2)} {away_team}"
    return prediction

In [ ]:
def export_all_predictions():

    listTeams = df_final["Home team name"].unique()
    for home in listTeams:
        for away in listTeams:
            if away != home:
                prediction = str(match_prediction(home, away))
                with open("all_matchs_predictions.txt", 'a') as file:
                    file.write(prediction + "\n")
    
    
    
export_all_predictions()

## OBSERVATIONS AND ANALYSIS

After conducting predictions using advanced statistics and machine learning techniques, we analyzed the results to gain insights into the performance of the predictive models. Here are some key observations:

The predicted scores generally show realistic values for most matches. While the actual and predicted scores do not match in most cases, it is important to recognize that football is a sport filled with surprises, and predicting precise scores can be challenging. Despite this, the predicted scores tend to align well with what we might reasonably expect from the teams' performance.

The distribution of predicted scores closely matches the real score distribution. In most matches, the model captures the typical scoring patterns observed in the Premier League. However, there are instances where extreme values, such as matches with more than 4 or 5 goals, might deviate from the actual outcomes. These high-scoring matches are less frequent and harder to predict accurately.

The percentages of predicted home wins, away wins, and draws align well with the actual percentages observed in the Premier League. The model effectively captures the probabilities of different outcomes, reflecting the teams' relative strengths and the likelihood of specific results.

## WAYS TO IMPROVE THE PERFORMANCE

I think that more data would not help much as the learning curves suggest., there are several ways we can further enhance their performance and accuracy. Here are some strategies to improve the model's predictive capabilities:

1. Feature Engineering:
Consider exploring additional relevant features that could provide valuable insights into team performance. It may involve incorporating historical performance data, player-specific statistics, or match context features.

2. Model Selection and Tuning:
Experiment with different machine learning algorithms, including various regression techniques and ensemble models, to find the best-fitted approach for predicting match outcomes.

3. Include Contextual Information:
Incorporate additional contextual information such as weather conditions, team injuries, recent form, or match location to capture external factors that may influence match outcomes.

4. Time-Series Analysis:
Consider treating the match data as a time-series to account for the dynamic nature of team performance over the course of a season. Time-series analysis techniques, such as autoregression or seasonality modeling, could provide more accurate predictions by capturing trends and patterns in team performance over time.

